## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 📊 Agent with Foundry Tracing

## Industry Use Case: Trade Execution Monitoring

This notebook demonstrates how to set up **telemetry in Microsoft Foundry** for custom agents.

| Feature | FSI Application |
|---------|-----------------|
| **Application Insights** | Monitor trade execution latency |
| **Trace IDs** | Track order flow across systems |
| **Live Metrics** | Real-time transaction monitoring |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")
print(f"Project Endpoint: {os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')[:50]}...")

In [ ]:
import asyncio
from random import uniform
from agent_framework.azure import AzureAIAgentClient
from agent_framework.observability import create_resource, enable_instrumentation, get_tracer
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id

print("✅ All imports loaded")

## Define Trade Execution Tool

In [ ]:
async def execute_trade(symbol: str, quantity: int, action: str) -> str:
    """Execute a stock trade."""
    await asyncio.sleep(uniform(0.1, 0.5))  # Simulate network latency
    price = round(uniform(100, 500), 2)
    total = round(price * quantity, 2)
    return f"{action.upper()} {quantity} shares of {symbol.upper()} at ${price} = ${total}"


async def get_market_price(symbol: str) -> str:
    """Get current market price for a symbol."""
    await asyncio.sleep(uniform(0.05, 0.2))  # Simulate API call
    prices = {"MSFT": 425.50, "AAPL": 178.25, "GOOGL": 142.80}
    price = prices.get(symbol.upper(), round(uniform(50, 300), 2))
    change = round(uniform(-5, 5), 2)
    return f"{symbol.upper()}: ${price} ({'+' if change > 0 else ''}{change}%)"


print("✅ Tools defined: execute_trade, get_market_price")

## Configure Foundry Tracing

This sets up telemetry to send traces to Application Insights attached to your Azure AI project.

In [ ]:
async def run_trade_agent_with_tracing():
    """Run trade execution agent with Foundry tracing."""
    print("\n--- Trade Execution Agent with Foundry Tracing ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(
            endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            credential=credential
        ) as project_client,
    ):
        # Get Application Insights connection string from Foundry
        try:
            conn_string = await project_client.telemetry.get_application_insights_connection_string()
            print(f"✅ Application Insights connected")
        except Exception as e:
            print(f"⚠️ No Application Insights found: {e}")
            print("Continuing without Azure Monitor (traces will go to console)...\n")
            conn_string = None
        
        # Configure Azure Monitor if connection string available
        if conn_string:
            configure_azure_monitor(
                connection_string=conn_string,
                enable_live_metrics=True,
                resource=create_resource(),
                enable_performance_counters=False,
            )
        
        # Enable instrumentation (including sensitive data for demo)
        enable_instrumentation(enable_sensitive_data=True)
        print("✅ Observability configured\n")
        
        # Trade requests
        requests = [
            "What's the current price of MSFT?",
            "Buy 100 shares of MSFT",
        ]
        
        # Run with tracing span
        with get_tracer().start_as_current_span("Trade Session", kind=SpanKind.CLIENT) as span:
            trace_id = format_trace_id(span.get_span_context().trace_id)
            print(f"🔍 Trace ID: {trace_id}")
            print("(View in Azure Portal > Application Insights > Transaction Search)\n")
            
            async with AzureAIAgentClient(credential=credential).as_agent(
                name="TradeAgent",
                instructions="You are a trading assistant. Help execute trades and check prices.",
                tools=[execute_trade, get_market_price],
            ) as agent:
                for request in requests:
                    print(f"User: {request}")
                    result = await agent.run(request)
                    print(f"TradeAgent: {result.text}\n")

await run_trade_agent_with_tracing()

## Key Takeaways

| Feature | Description |
|---------|-------------|
| `AIProjectClient.telemetry` | Get App Insights connection from Foundry |
| `configure_azure_monitor()` | Set up Azure Monitor exporter |
| `enable_instrumentation()` | Enable trace collection |
| `get_tracer().start_as_current_span()` | Create custom trace spans |

## View Traces in Azure Portal

1. Go to **Azure Portal** > **Application Insights**
2. Select **Transaction Search**
3. Filter by the Trace ID shown above